In [ ]:
import os
import csv
import sys
import tensorflow as tf
import keras
import keras.preprocessing.image
import json
import random
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import math

In [ ]:
from keras.callbacks import Callback

In [ ]:
# keras_retinanet imports
from keras_retinanet import models
from keras_retinanet import losses
from keras_retinanet import layers
from keras_retinanet.models.retinanet import retinanet_bbox

from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.callbacks.eval import Evaluate
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
# sys.path.append('/root/amol/product_detection/keras-retinanet/keras_retinanet/preprocessing/')
# from csv_generator import CSVGenerator
# from ..models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet
# from keras_retinanet.models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet


from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.utils.keras_version import check_keras_version
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from eval_modified import evaluate

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

In [ ]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# TRAIN VAL SPLIT

In [ ]:
# # load annotations
# dataset = []
# with open('/root/data/lice_detection/lice_dataset_fish_only.csv', 'r') as f:
#     reader = csv.reader(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)
#     for row in reader:
#         dataset.append(row)

In [ ]:
# dataset = sorted(dataset, key=lambda k:k[0])

In [ ]:
# with open('/root/data/lice_detection/lice_dataset_fish_only_train.csv', 'w') as csvfile:
#     writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
#     for ts in dataset[:390]:
#         writer.writerow(ts)

In [ ]:
# with open('/root/data/lice_detection/lice_dataset_fish_only_val.csv', 'w') as csvfile:
#     writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
#     for ts in dataset[390:]:
#         writer.writerow(ts)

In [ ]:
annotations = '/root/data/lice_detection/lice_dataset_fish_only_train.csv'
classes = '/root/data/lice_detection/class_id.csv'
batch_size = 4
val_annotations = '/root/data/lice_detection/lice_dataset_fish_only_val.csv'

# create generators

In [ ]:
def create_generators(annotations,classes,batch_size=1,val_annotations=''):
    # create random transform generator for augmenting training data
    transform_generator = random_transform_generator(flip_x_chance=0.5)
    
#     transform_generator = random_transform_generator(
#     min_rotation=-0.2,
#     max_rotation=0.2,
#     min_translation=(-0.3, -0.3),
#     max_translation=(0.3, 0.3),
#     min_shear=-0.3,
#     max_shear=0.3,
#     min_scaling=(0.5, 0.5),
#     max_scaling=(1.3, 1.3),
#     flip_x_chance=0,
#     flip_y_chance=0.5)
    
    
    train_generator = CSVGenerator(
        annotations,
        classes,
        transform_generator=transform_generator,
        batch_size=batch_size,
        image_min_side=800,
        image_max_side=1500
    )


    if val_annotations:
        validation_generator = CSVGenerator(
            val_annotations,
            classes,
            batch_size=batch_size,
            image_min_side=800,
            image_max_side=1500
        )
    else:
        validation_generator = None
    

    return train_generator, validation_generator

In [ ]:
train_generator, validation_generator = create_generators(annotations, classes, batch_size, val_annotations)

In [ ]:
train_generator.size()

In [ ]:
validation_generator.size()

# create model

In [ ]:
def model_with_weights(model, weights, skip_mismatch):
    if weights is not None:
        model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
    return model

In [ ]:
def create_models(backbone_retinanet, num_classes, weights, multi_gpu=0, freeze_backbone=False):
    modifier = freeze_model if freeze_backbone else None

    # Keras recommends initialising a multi-gpu model on the CPU to ease weight sharing, and to prevent OOM errors.
    # optionally wrap in a parallel model
    if multi_gpu > 1:
        with tf.device('/cpu:0'):
            model = model_with_weights(backbone_retinanet(num_classes, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = multi_gpu_model(model, gpus=multi_gpu)
    else:
        model          = model_with_weights(backbone_retinanet(num_classes, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = model

    # make prediction model
    prediction_model = retinanet_bbox(model=model)

    # compile model
    training_model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )

    return model, training_model, prediction_model

In [ ]:
backbone = models.backbone('resnet50')
weights = backbone.download_imagenet()

In [ ]:
model, training_model, prediction_model = create_models(
    backbone_retinanet=backbone.retinanet,
    num_classes=train_generator.num_classes(),
    weights=weights
)

In [ ]:
# recall, precision, mAP = evaluate(validation_generator, prediction_model)

# train model

In [ ]:
import json

In [ ]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 1e-5
    drop = 0.5
    epochs_drop = 20.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
# save model
filepath = os.path.join('/root/data/models/imenco/detection/', 'model_{epoch:02d}.h5')
checkpoint = ModelCheckpoint(filepath)

In [ ]:
# create history callback
class SaveHistory(Callback):
    
    def __init__(self, json_path):
        self.json_path = json_path
    
    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        with open(self.json_path, 'w') as f:
            json.dump(self.history, f)
        


In [ ]:
saveh = SaveHistory('./history.json')

In [ ]:
# start training
history = training_model.fit_generator(
        generator=train_generator,
        steps_per_epoch=500//batch_size,
        epochs=40,
        verbose=1,
        validation_data= validation_generator,
        validation_steps= 50 // batch_size,
        callbacks=[lr_scheduler, saveh]
    )

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
training_model.save('tr_overfit.h5')

In [ ]:
prediction_model.save('pred_overfit.h5')